In [ ]:
import sys
# setting path
sys.path.append('..')
# importing
import ECC_library
from qiskit.circuit import Gate
from qiskit import ClassicalRegister, QuantumCircuit, QuantumRegister
from qiskit import Aer, execute, IBMQ, BasicAer
from qiskit.aqua import QuantumInstance
from qiskit.visualization import plot_bloch_multivector, plot_histogram

In [ ]:
a=4
b=5
p=7
n=4
g_s='0'
def test(a,b,p,n,g_s):
    x = QuantumRegister(n,name='x')
    y = QuantumRegister(n,name='y')
    z = QuantumRegister(n,name='z')
    c = QuantumRegister(1,name='d')
    g = QuantumRegister(1,name='g')
    clas_a = ClassicalRegister(n)
    clas_b = ClassicalRegister(n)
    clas_z = ClassicalRegister(n)
    clas_c = ClassicalRegister(1)
    clas_g = ClassicalRegister(1)
    circuit = QuantumCircuit(x,y,z,c,g,clas_a,clas_b,clas_z,clas_c,clas_g)
    s_a=bin(int(a))[2:].zfill(n)
    s_b=bin(int(b))[2:].zfill(n)
    s_p=bin(int(p))[2:].zfill(n)
    s_p=s_p[::-1]
    if(g_s=='1'):
        circuit.x(g)
    for i in range(0, n):
        if s_a[i] == '1':
            circuit.x(x[n-1-i])
        if s_b[i] == '1':
            circuit.x(y[n-1-i])



    for i in range(n-1,0,-1):
        ECC_library.controlled_addition_modp(circuit,x[i:i+1],y,z,c,g,s_p,n)
        ECC_library.doubling_modp(circuit,z,c,g,s_p,n)
    ECC_library.controlled_addition_modp(circuit,x[0:1],y,z,c,g,s_p,n)



    for i in range(0, n):
        circuit.measure(x[i],clas_a[i])
    for i in range(0, n):
        circuit.measure(y[i],clas_b[i])
    for i in range(0, n):
        circuit.measure(z[i],clas_z[i])
    circuit.measure(c,clas_c)
    circuit.measure(g,clas_g)
    print('a:',a,'b:',b,'p:',p,'g:',g_s)
    backend = BasicAer.get_backend('qasm_simulator')
    simulation = execute(circuit, backend=backend ,shots=10)
    sim_result = simulation.result()
    out=list(sim_result.get_counts().keys())[0]
    output=out.split(' ')
    output_a = output[4]
    output_b = output[3]
    output_z = output[2]
    output_c = output[1]
    res_g = output[0]
    res_a = int(output_a, 2)
    res_b = int(output_b, 2)
    res_z = int(output_z, 2)
    res_c = int(output_c, 2)
    print('res_a:',res_a,'res_b:',res_b,'res_z:',res_z,'res_c:',res_c,'res_g:',res_g)
    if (res_z!=(a*b)%p):
        print('error!')

In [ ]:
IBMQ.load_account();
for n in range (4,14):
    print('n:',n)
    d_vec=[]
    for p in [7,11,13,17,19,23,29,31,43,47]:
        if(p<2**n):
            #print('p:',p)
            a=2
            b=3
            d=test(a,b,p,n,'0')
            d_vec.append(d)
            #test(a,b,p,n,'1')
    print(d_vec)
    print('--------')